In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from globalModule.globalModule import *

In [ ]:
__name__ = "RNN"

# Model

In [137]:
def getModel():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))

    model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])


    return model

In [142]:
def getData():
    labelMap = {label:num for num, label in enumerate(actions)}
    sequences, labels = [], []
    for action in actions:
        for sequence in np.array(os.listdir(os.path.join(gesturesDataPath, action))).astype(int):
            window = []
            for frame_num in range(sequenceLen):
                try:
                    res = np.load(os.path.join(gesturesDataPath, action, str(sequence), "{}.npy".format(frame_num)))
                    window.append(res)
                except Exception as e:
                    print(e)
            sequences.append(window)
            labels.append(labelMap[action])
    
    return sequences, labels 

In [143]:
sequences, labels = getData()
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [146]:
actions.shape[0]

6

In [144]:
### INITS

tb_callback = TensorBoard(log_dir=log_dir)

In [147]:
model = getModel()
model.fit(X_train, y_train, epochs=80, callbacks=[tb_callback])

Epoch 1/400
71/71 [==============================] - 9s 59ms/step - loss: 0.4681 - categorical_accuracy: 0.3050
Epoch 2/400
71/71 [==============================] - 5s 76ms/step - loss: 0.3072 - categorical_accuracy: 0.5365
Epoch 3/400
71/71 [==============================] - 6s 79ms/step - loss: 0.2491 - categorical_accuracy: 0.6430
Epoch 4/400
71/71 [==============================] - 5s 69ms/step - loss: 0.2261 - categorical_accuracy: 0.6818
Epoch 5/400
71/71 [==============================] - 5s 67ms/step - loss: 0.1951 - categorical_accuracy: 0.7328
Epoch 6/400
71/71 [==============================] - 5s 72ms/step - loss: 0.1986 - categorical_accuracy: 0.7394
Epoch 7/400
71/71 [==============================] - 6s 79ms/step - loss: 0.1992 - categorical_accuracy: 0.7386
Epoch 8/400
71/71 [==============================] - 6s 78ms/step - loss: 0.1724 - categorical_accuracy: 0.7790
Epoch 9/400
71/71 [==============================] - 6s 81ms/step - loss: 0.1548 - categorical_accuracy:

In [ ]:

print(model.summary())

In [ ]:
model.save(f"{modelPath}/{__name__}/test.h5")